In [1]:
import os
import json
import datetime

def train():
    # 初始化参数
    trans_prob = {}  # 转移概率
    emit_prob = {}  # 发射概率
    init_prob = {}  # 状态出现次数
    Count_dict={}
    state_list = ['B', 'M', 'E', 'S']
    for state in state_list:
        trans = {}
        for s in state_list:
            trans[s] = 0
        trans_prob[state] = trans
        emit_prob[state] = {}
        init_prob[state] = 0
        Count_dict[state] = 0
    count = -1
    # 读取并处理单词、计算概率矩阵
    path = '实训专题1/trainCorpus.txt'
    for line in open(path, 'r',  encoding="gbk"):
        count += 1
        line = line.strip()
        if not line:
            continue

        # 读取每一行的单词
        word_list = []
        for i in line:
            if i != ' ':
                word_list.append(i)

        # 标注每个单词的位置标签
        word_label = []
        for word in line.split():
            label = []
            if len(word) == 1:
                label.append('S')
            else:
                label += ['B'] + ['M'] * (len(word) - 2) + ['E']
            word_label.extend(label)

        # 统计各个位置状态下的出现次数，用于计算概率
        for index, value in enumerate(word_label):
            Count_dict[value] += 1
            if index == 0:
                init_prob[value] += 1
            else:
                trans_prob[word_label[index - 1]][value] += 1
                emit_prob[word_label[index]][word_list[index]] = (
                        emit_prob[word_label[index]].get(
                            word_list[index], 0) + 1.0)
    # 初始概率
    for key, value in init_prob.items():
        init_prob[key] = value * 1 / count
        # 转移概率
    for key, value in trans_prob.items():
        for k, v in value.items():
            value[k] = v / Count_dict[key]
        trans_prob[key] = value
    # 发射概率，采用加1平滑
    for key, value in emit_prob.items():
        for k, v in value.items():
            value[k] = (v + 1) / Count_dict[key]
        emit_prob[key] = value
    # 将3个概率矩阵保存至json文件
    model = 'hmm_model.json'
    # 若文件已存在，则删除
    if os.path.exists(model):
        os.remove(model)
    f = open(model, 'a+')
    f.write(json.dumps(trans_prob) + '\n' + json.dumps(emit_prob) +
            '\n' + json.dumps(init_prob))
    f.close()


In [2]:
def viterbi(text, state_list, init_prob, trans_prob, emit_prob):
    V = [{}]
    path = {}
    # 初始概率
    for state in state_list:
        V[0][state] = init_prob[state] * emit_prob[state].get(text[0], 0)
        path[state] = [state]

    # 当前语料中所有的字
    key_list = []
    for key, value in emit_prob.items():
        for k, v in value.items():
            key_list.append(k)

    # 计算待分词文本的状态概率值，得到最大概率序列
    for t in range(1, len(text)):
        V.append({})
        newpath = {}
        for state in state_list:
            if text[t] in key_list:
                emit_count = emit_prob[state].get(text[t], 0)
            else:
                emit_count = 1
            (prob, a) = max(
                [(V[t - 1][s] * trans_prob[s].get(state, 0)* emit_count, s)
                             for s in state_list if V[t - 1][s] > 0])
            V[t][state] = prob
            newpath[state] = path[a] + [state]
        path = newpath
    # 根据末尾字的状态，判断最大概率状态序列
    if emit_prob['M'].get(text[-1], 0) > emit_prob['S'].get(text[-1], 0):
        (prob, a) = max([(V[len(text) - 1][s], s) for s in ('E', 'M')])
    else:
        (prob, a) = max([(V[len(text) - 1][s], s) for s in state_list])

    return (prob, path[a])

In [3]:
def cut(text):
    state_list = ['B', 'M', 'E', 'S']
    model = 'hmm_model.json'
    # 先检查当前路径下是否有json文件，如果有json文件，需要删除
    if os.path.exists(model):
        f = open(model, 'rb')
        trans_prob = json.loads(f.readline())
        emit_prob = json.loads(f.readline())
        init_prob = json.loads(f.readline())
        f.close()
    else:
        trans_prob = {}
        emit_prob = {}
        init_prob = {}
    # 利用维特比算法，求解最大概率状态序列
    prob, pos_list = viterbi(text, state_list, init_prob, trans_prob, emit_prob)
    # 判断待分词文本每个字的状态，输出结果
    begin, follow = 0, 0
    for index, char in enumerate(text):
        state = pos_list[index]
        if state == 'B':
            begin = index
        elif state == 'E':
            yield text[begin: index+1]
            follow = index + 1
        elif state == 'S':
            yield char
            follow = index + 1
    if follow < len(text):
        yield text[follow:]


In [4]:
# 训练、分词
starttime = datetime.datetime.now()
train()
endtime = datetime.datetime.now()
print((endtime-starttime).seconds)

text = '深航客机攀枝花机场遇险：机腹轮胎均疑受损，跑道灯部分损坏'
cut(text)
print(text)
print(str(list(cut(text))))

0
深航客机攀枝花机场遇险：机腹轮胎均疑受损，跑道灯部分损坏
['深航', '客机', '攀枝', '花机场', '遇险', '：', '机腹', '轮胎', '均', '疑受', '损，', '跑', '道灯', '部分', '损坏']


In [5]:
import jieba
def word_extract():
    # 读取文件
    corpus = []
    path = '实训专题1/flightnews.txt'
    content = ''
    for line in open(path, 'r', errors='ignore'):
        line = line.strip()
        content += line
    corpus.append(content)
    # 加载停用词
    stop_words = []
    path = '实训专题1/stopword.txt'
    for line in open(path, encoding='utf8'):
        line = line.strip()
        stop_words.append(line)
        # jieba分词
    split_words = []
    word_list = jieba.cut(corpus[0])
    for word in word_list:
        if word not in stop_words:
            split_words.append(word)
    # 提取前10个高频词
    dic = {}
    word_num = 10
    for word in split_words:
        dic[word] = dic.get(word, 0) + 1
    freq_word = sorted(dic.items(), key = lambda x: x[1],
                       reverse=True) [: word_num]
    print('样本：' + corpus[0])
    print('样本分词效果：' + '/ '.join(split_words))
    print('样本前10个高频词：' + str(freq_word))


In [6]:
word_extract()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\admin\AppData\Local\Temp\jieba.cache
Loading model cost 0.621 seconds.
Prefix dict has been built successfully.


样本：原标题：深航飞机攀枝花遇险官方报告：机长私自上跑道检查还迟报 来源：上游新闻上游新闻持续关注的“深圳航空ZH9247航班10月16日在四川攀枝花机场降落时发生跑道外接地不安全事件”有了最新进展——10月19日深夜，民航西南管理局通过媒体公布了初步调查报告。据上述报告披露，此次事件造成飞机机腹出现两处大小为20*12*3.2、45*5*贯穿（厘米）的损伤，客机面临“机毁人亡”风险；航班机长存在未经机场许可前往运行中的跑道、并在1小时后才向塔台报告等情况。民航业内人士告诉上游新闻记者，深圳航空当事机组降落阶段处置是否合规、损伤是否构成飞机报废条件等情况，还需要民航监管机构进一步调查，但飞机落地后机长未经许可前往跑道、落地一小时后才向塔台报告的行为，已经严重违反了民航相关安全规定。资深飞行员：距离机毁人亡，仅一步之遥上游新闻此前刊发《深航客机攀枝花机场遇险：机腹轮胎均疑受损，跑道灯部分损坏》、《深航回应航班攀枝花“航母机场”遇险被质疑避重就轻：只说扎胎不讲机腹受损》报道显示，10月16日，由西安咸阳飞往攀枝花保安营机场的深圳航空ZH9247航班在攀枝花机场落地时发生不安全事件，“落地阶段疑似机腹擦挂跑道监控天线，起落架提前接地，到位后检查轮胎有扎伤痕迹。”攀枝花机场事后对跑道、灯光等相关设施进行全面检查，“航向台监控天线被撞断、6个进近灯被撞坏”。深圳航空官方事后仅在微博简单回应称，该航班是飞机轮胎被扎导致后续航班取消，未披露更多的详细情况。深航的这一回应，被舆论质疑为避重就轻、选择性披露。10月19日深夜，民航西南局通过四川当地媒体《川观新闻》，披露了10月16日深航ZH9247航班在攀枝花机场跑道外接地事件的初步调查报告。报告认定该航班在降落时，出现了“跑道外接地的不安全事件”。资料显示，“跑道外接地”是指客机在落地时没有在标准硬化跑道上落地，在跑道外同地面发生接触，使飞机有可能陷入跑道外的地面，容易造成飞机机身倾斜、撕裂、起落架设备受损等严重后果。国内某航空公司资深飞行员赖先生对上游新闻记者表示，在攀枝花这样的山顶机场发生跑道外接地事件，“距离机毁人亡，仅一步之遥。”机腹现两处损伤，最严重可致飞机报废初步调查报告披露，当天航班上共计99人，含3名飞行机组人员、6名客舱机组人员、1名跟机机务和89名旅客。初步调查报告也披露了10月16日执飞航班的B-8667空中